# Imports

In [ ]:
from getpass import getpass

token = getpass('Enter your GitHub personal access token: ')
name = getpass('Enter your GitHub name: ')

!git clone https://{token}@github.com/{name}/comp_med.git
#%cd comp_med


Enter your GitHub personal access token: ··········
Enter your GitHub name: ··········
Cloning into 'comp_med'...
remote: Enumerating objects: 179, done.
remote: Counting objects: 100% (179/179), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 179 (delta 105), reused 81 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (179/179), 1.02 MiB | 4.25 MiB/s, done.
Resolving deltas: 100% (105/105), done.


In [ ]:
#!rm -r comp_med

## Make Code Deterministic for Reproducibility

In [3]:
import os
import random
import numpy as np
import torch
def set_seed(SEED):
  os.environ["PYTHONHASHSEED"] = str(SEED)
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed_all(SEED)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [4]:
!pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 33.2 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [5]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
import numpy as np
import torch
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import json
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
)

#my imports
from comp_med.models.attentionCNN import CNN_1D
from comp_med.models.oldCNN import CNN_2D
from comp_med.data.preprocessing import get_dataloaders


## Save Function

In [42]:
def save_file(object_to_save = None, save_name=None, save_dir = '/content/drive/MyDrive/logs'):
  print(object_to_save)
  os.makedirs(save_dir, exist_ok=True)
  out_path = os.path.join(save_dir, save_name)
  with open(out_path, 'w') as f:
      json.dump(object_to_save, f, indent=2)

## Training Helpers

In [7]:
def train(model, criterion, optimizer, train_loader, val_loader, epochs, device="cpu", return_loss=False):
  model.to(device)
  loss_tracker = []
  for epoch in range(epochs):
    pbar = tqdm(train_loader, desc=f"Train the model in epoch {epoch}...")
    loss = 0
    for x,y in pbar:
      optimizer.zero_grad()
      x,y = x.to(device),y.to(device)
      out = model(x)
      out = torch.sigmoid(out)
      loss = criterion(out,y)
      loss.backward()
      optimizer.step()
      loss = loss.item()
      pbar.set_description(f"Current loss in epoch {epoch} is {loss}")
      loss_tracker.append(loss)
    #start validation
    acc = eval(model, val_loader, device)
    print(f"Acc on val in epoch {epoch} is: {acc}")
  if return_loss:
    return loss_tracker

def eval(model, data_loader, device="cpu", all_metrics=False):
    model.to(device)
    #start validation
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
      for x,y in data_loader:
        x,y = x.to(device), y.to(device)
        out = model(x)
        preds.append((torch.sigmoid(out) > 0.5).float())
        labels.append(y)

    model.train()
    preds = torch.cat(preds).cpu().numpy().ravel()
    labels = torch.cat(labels).cpu().numpy().ravel()
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    sens = recall_score(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    spec = tn/(tn + fp)
    f1 = f1_score(labels, preds)
    if all_metrics:
      return {"acc":acc, "prec":prec, "sens":sens, "spec":spec, "f1":f1}
    else:
      return acc

# Verification Dummy Task

In [8]:
# create a dummy dataset to verify the architecture works
batch_size = 256
length = 5
samples_per_sec = 1000
num_leads = 12
num_samples = 500

t = np.arange(0, length, 1/samples_per_sec)
base_freq = 1

X = np.zeros((num_samples, num_leads, t.size))
labels = np.zeros((num_samples,))

for i in range(num_samples):
  label = i % 2
  for l in range(num_leads):
    amplitude = l+1
    X[i,l] = amplitude * np.sin( 2*np.pi * base_freq * t)
  if label == 1:
    #get a random lead
    lead_idx = np.random.randint(0, num_leads)
    #doulbe the frequency when label is 1
    X[i, lead_idx] = (lead_idx+1) * np.sin( 2*np.pi * base_freq*2 * t)

  #add some noise
  noise = np.random.normal(0, 0.1, size=X[i].shape)
  X[i] = X[i]+noise
  labels[i]= label

X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(labels).unsqueeze(1).float()

len = X_t.shape[0]
permutation = torch.randperm(len)
train_idx = permutation[:400]
test_idx =permutation[400:]

X_train, y_train = X_t[train_idx], y_t[train_idx]
X_test, y_test = X_t[test_idx] ,y_t[test_idx]
print(X_t.shape)
print(X_test.shape)
print(X_train.shape)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = batch_size)

torch.Size([500, 12, 5000])
torch.Size([100, 12, 5000])
torch.Size([400, 12, 5000])


In [9]:
epochs = 25
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNN_1D()
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()
train(model, criterion, optimizer, train_loader, train_loader ,epochs, device)
print("Fianl acc on test is: ", eval(model, test_loader, device))

Current loss in epoch 0 is 0.7298088073730469: 100%|██████████| 2/2 [00:01<00:00,  1.03it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Acc on val in epoch 0 is: 0.4775


Current loss in epoch 1 is 0.40963423252105713: 100%|██████████| 2/2 [00:00<00:00,  7.58it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Acc on val in epoch 1 is: 0.4775


Current loss in epoch 2 is 0.12633655965328217: 100%|██████████| 2/2 [00:00<00:00,  7.53it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


...

Acc on val in epoch 22 is: 1.0


Current loss in epoch 23 is 0.0037121449131518602: 100%|██████████| 2/2 [00:00<00:00,  7.53it/s]


Acc on val in epoch 23 is: 1.0


Current loss in epoch 24 is 0.003171222750097513: 100%|██████████| 2/2 [00:00<00:00,  7.55it/s]


Acc on val in epoch 24 is: 1.0
Fianl acc on test is:  1.0


# Convergence Analysis of the Model

In [10]:
train_loader, val_loader, test_loader = get_dataloaders("/content/drive/MyDrive/ptbdb",preprocessed_data_path="/content/drive/MyDrive/ptbdb/preprocessed_data.pt", train_ratio=0.6, val_ratio=0.1)

After filtering, we got: 228 records. Healthy: 80, Disease: 148
Patients: train: 120 | val: 20 | test: 60
Load data from given path


In [11]:
epochs = 20
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
set_seed(0)
model = CNN_1D()
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCELoss()
losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device, return_loss=True)
print("Fianl acc on test is: ", eval(model, test_loader, device))

Current loss in epoch 0 is 0.10174188017845154: 100%|██████████| 23/23 [00:05<00:00,  4.25it/s]


Acc on val in epoch 0 is: 0.6530782029950083


Current loss in epoch 1 is 0.0340830460190773: 100%|██████████| 23/23 [00:05<00:00,  4.25it/s]


Acc on val in epoch 1 is: 0.6589018302828619


Current loss in epoch 2 is 0.01336760725826025: 100%|██████████| 23/23 [00:05<00:00,  4.26it/s]


...

Acc on val in epoch 17 is: 0.7054908485856906


Current loss in epoch 18 is 0.00023154637892730534: 100%|██████████| 23/23 [00:05<00:00,  4.19it/s]


Acc on val in epoch 18 is: 0.7063227953410982


Current loss in epoch 19 is 0.00016317047993652523: 100%|██████████| 23/23 [00:05<00:00,  4.23it/s]


Acc on val in epoch 19 is: 0.7054908485856906
Fianl acc on test is:  0.9325691166554282


In [13]:
print(losses)

[0.6905691623687744, 0.786510705947876, 0.7059726715087891, 0.6001118421554565, 0.47714635729789734, 0.4209638833999634, 0.3082520067691803, 0.2662844657897949, 0.2587848901748657, 0.28281304240226746, 0.1781378537416458, 0.19684909284114838, 0.1342707872390747, 0.15646979212760925, 0.11259046941995621, 0.11482617259025574, 0.13019491732120514, 0.13390281796455383, 0.10805634409189224, 0.08212548494338989, 0.09083904325962067, 0.09195514023303986, 0.10174188017845154, 0.12215659022331238, 0.09455606341362, 0.06379201263189316, 0.08316996693611145, 0.0641225129365921, 0.08017149567604065, 0.06020168587565422, 0.07326832413673401, 0.07447246462106705, 0.048916351050138474, 0.038726530969142914, 0.03138140216469765, 0.049165062606334686, 0.06500668078660965, 0.03210264444351196, 0.03014521114528179, 0.04445832967758179, 0.02848118357360363, 0.03116218000650406, 0.024265948683023453, 0.021523119881749153, 0.021911637857556343, 0.0340830460190773, 0.017026258632540703, 0.030514370650053024,

# Sensitivity Analysis Towards Data Perturbation
We add random noise $\mathcal{N}(0,\sigma^2)$ to the input of the model and evaluate how robust the model is.

In [14]:
trials = 3
epochs = 20
lr = 1e-3
seeds = [0,1,2]

In [15]:
def eval_with_perturbation(model, data_loader, std_levels=[1e-3,5e-3,1e-2,5e-2,1e-1,5e-1,1], device="cpu"):
    accs = []
    for std in std_levels:
      model.to(device)
      #start validation
      model.eval()
      correct = 0
      total = 0
      with torch.no_grad():
        for x,y in data_loader:
          x,y = x.to(device), y.to(device)
          # add noise
          x = x + torch.randn_like(x) * std
          out = model(x)
          preds = (torch.sigmoid(out) > 0.5).float()
          correct += (preds == y).sum().item()
          total += y.shape[0]
      model.train()
      acc = correct / total
      accs.append(acc)
    return accs

In [16]:
trial_to_acc = {}
for trial,seed in zip(range(trials),seeds):
  set_seed(seed)
  model = CNN_1D()
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device)
  accs = eval_with_perturbation(model, test_loader, device=device)
  trial_to_acc[trial] = accs

Current loss in epoch 0 is 0.10174188017845154: 100%|██████████| 23/23 [00:05<00:00,  4.24it/s]


Acc on val in epoch 0 is: 0.6530782029950083


Current loss in epoch 1 is 0.0340830460190773: 100%|██████████| 23/23 [00:05<00:00,  4.24it/s]


Acc on val in epoch 1 is: 0.6589018302828619


Current loss in epoch 2 is 0.01336760725826025: 100%|██████████| 23/23 [00:05<00:00,  4.22it/s]


...

Acc on val in epoch 4 is: 0.7695507487520798


Current loss in epoch 5 is 0.005845550913363695: 100%|██████████| 23/23 [00:05<00:00,  4.20it/s]


Acc on val in epoch 5 is: 0.7321131447587355


Train the model in epoch 6...:   0%|          | 0/23 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
save_file(object_to_save=trial_to_acc, save_name="noise_perturbation.json")

# Sensetivity Analysis Towards Hyperparameters
We evaluate the sensitivity towards hyperparameters. We execute every experiment 3 times and report the average

In [ ]:
seeds = [0,0,0] #always use the same random seed for every run

## Kernel Size
We evaluate different kernel sizes  $[50,100,200]$

In [17]:
epochs = 20
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
kernel_sizes = [25,50,200]
kernel_to_acc = {}
seeds = [0,0,0]
for kernel_size, seed in zip(kernel_sizes,seeds):
  set_seed(seed)
  model = CNN_1D(kernel_size=kernel_size)
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device, return_loss=False)
  acc=eval(model, test_loader, device)
  print("Fianl acc on test is: ",acc )
  kernel_to_acc[kernel_size] = acc

Current loss in epoch 0 is 0.36201754212379456:  61%|██████    | 14/23 [00:03<00:02,  4.13it/s]


KeyboardInterrupt: 

In [ ]:

save_file(object_to_save=kernel_to_acc, save_name="kernel_size.json")

{25: 0.9086311530681052, 50: 0.9265003371544167, 200: 0.9369521240728254}


## Stride
We evaluate different strides $[25,50,100]$

In [19]:
strides = [25,100,200]
stride_to_acc = {}
seeds = [0,0,0]
for stride, seed in zip(strides,seeds):
  set_seed(seed)
  model = CNN_1D(stride = stride)
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device, return_loss=False)
  acc=eval(model, test_loader, device)
  print("Fianl acc on test is: ",acc )
  stride_to_acc[stride] = acc

Current loss in epoch 0 is 0.21232058107852936:  48%|████▊     | 11/23 [00:04<00:05,  2.21it/s]


KeyboardInterrupt: 

In [ ]:

save_file(object_to_save=stride_to_acc, save_name="strides.json")

{25: 0.897167902899528, 100: 0.9416722859069454, 200: 0.914025623735671}


## Attention Heads
We evaluate different amount of attention heads $[2,4,8]$

In [20]:
attention_heads = [1,2,10]
head_to_acc = {}
seeds = [0,0,0]
for heads,seed in zip(attention_heads,seeds):
  set_seed(seed)
  model = CNN_1D(attn_heads=heads)
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device, return_loss=False)
  acc=eval(model, test_loader, device)
  print("Fianl acc on test is: ",acc )
  head_to_acc[heads] = acc

Current loss in epoch 0 is 0.22072342038154602:  52%|█████▏    | 12/23 [00:02<00:02,  5.08it/s]


KeyboardInterrupt: 

In [ ]:

save_file(object_to_save=head_to_acc, save_name="attention_heads.json")

{10: 0.9204315576534052, 1: 0.928186109238031, 2: 0.928186109238831}


## Filter Size
We investigate different filter sizes $[10,20,40]$

In [21]:
filter_sizes = [4,12,40]
filter_to_acc = {}
seeds = [0,0,0]
for filter,seed in zip(filter_sizes,seeds):
  set_seed(seed)
  model = CNN_1D(filters=filter)
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device, return_loss=False)
  acc=eval(model, test_loader, device)
  print("Fianl acc on test is: ",acc )
  filter_to_acc[filter] = acc

Current loss in epoch 0 is 0.5765625834465027:  74%|███████▍  | 17/23 [00:02<00:00,  7.31it/s]


KeyboardInterrupt: 

In [ ]:

save_file(object_to_save=filter_to_acc, save_name="filters.json")

{4: 0.899527983816588, 12: 0.9359406608226568, 40: 0.9221173297370195}


## 3-Lead Ablation

In [23]:
train_loader, val_loader, test_loader = get_dataloaders("/content/drive/MyDrive/ptbdb",preprocessed_data_path="/content/drive/MyDrive/ptbdb/preprocessed_data_3leads.pt", desired_leads=['i','ii','iii'],train_ratio=0.6, val_ratio=0.1)

After filtering, we got: 228 records. Healthy: 80, Disease: 148
Patients: train: 120 | val: 20 | test: 60
Load data from given path


In [24]:
trial_to_acc = {}
seeds = [0,1,2]
trials = 3
lr = 0.001
epochs = 20
device = "cuda" if torch.cuda.is_available() else "cpu"
for trial,seed in zip(range(trials),seeds):
  set_seed(seed)
  model = CNN_1D(num_leads = 3)
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device)
  accs = eval(model, test_loader, device=device)
  trial_to_acc[trial] = accs

Current loss in epoch 0 is 0.21026206016540527: 100%|██████████| 23/23 [00:02<00:00,  9.16it/s]


Acc on val in epoch 0 is: 0.6530782029950083


Current loss in epoch 1 is 0.18960320949554443: 100%|██████████| 23/23 [00:02<00:00,  9.24it/s]


Acc on val in epoch 1 is: 0.6530782029950083


Current loss in epoch 2 is 0.06585603207349777: 100%|██████████| 23/23 [00:02<00:00,  9.22it/s]


...

Acc on val in epoch 17 is: 0.891846921797005


Current loss in epoch 18 is 0.0030544346664100885: 100%|██████████| 23/23 [00:02<00:00,  9.21it/s]


Acc on val in epoch 18 is: 0.8926788685524126


Current loss in epoch 19 is 0.001591993379406631: 100%|██████████| 23/23 [00:02<00:00,  9.22it/s]


Acc on val in epoch 19 is: 0.8968386023294509


In [ ]:

save_file(object_to_save=trial_to_acc, save_name="3_lead_experiment.json")

{0: 0.9207687120701281, 1: 0.9238031018206339, 2: 0.9177343223196224}


In [28]:

print("Average is: ", (trial_to_acc[0] + trial_to_acc[1] + trial_to_acc[2]) / 3)

Average is:  0.9275118004045853


In [29]:
train_loader, val_loader, test_loader = get_dataloaders("/content/drive/MyDrive/ptbdb",preprocessed_data_path="/content/drive/MyDrive/ptbdb/preprocessed_data.pt",train_ratio=0.6, val_ratio=0.1)

After filtering, we got: 228 records. Healthy: 80, Disease: 148
Patients: train: 120 | val: 20 | test: 60
Load data from given path


In [30]:
trial_to_acc = {}
seeds = [0,1,2]
trials = 3
lr = 0.001
epochs = 20
device = "cuda" if torch.cuda.is_available() else "cpu"
for trial,seed in zip(range(trials),seeds):
  set_seed(seed)
  model = CNN_1D(num_leads = 12)
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, val_loader ,epochs, device)
  accs = eval(model, test_loader, device=device)
  trial_to_acc[trial] = accs

Current loss in epoch 0 is 0.10174188017845154: 100%|██████████| 23/23 [00:05<00:00,  4.21it/s]


Acc on val in epoch 0 is: 0.6530782029950083


Current loss in epoch 1 is 0.06020168587565422:  30%|███       | 7/23 [00:01<00:03,  4.08it/s]


KeyboardInterrupt: 

In [31]:

save_file(object_to_save=trial_to_acc, save_name="12_lead_experiment.json")

{}


In [32]:
print("Average is: ", (trial_to_acc[0] + trial_to_acc[1] + trial_to_acc[2]) / 3)

KeyError: 0

# 5-Fold Cross Validation
In this section we compare our model to the 2D CNN model on 5-Fold Cross Validation

In [33]:
seeds = [0,0,0]
#my imports
from comp_med.data.preprocessing import get_record_paths, filter_records, split_patients

#get patient data and split on my own
path="/content/drive/MyDrive/ptbdb"
records = get_record_paths(path)
filtered_records = filter_records(records)
#now obtain k-fold split
train_folds, test_folds = split_patients(records, k_fold=5)

After filtering, we got: 228 records. Healthy: 80, Disease: 148
k_fold cv has no validation set


In [34]:
epochs = 20
lr = 1e-3
device = "cuda" if torch.cuda.is_available() else "cpu"

In [35]:
k_fold_acc = {}
for i, train_ids, test_ids in zip(range(5),train_folds,test_folds):
  set_seed(0)
  train_loader,val_loader, test_loader = get_dataloaders("/content/drive/MyDrive/ptbdb",preprocessed_data_path="/content/drive/MyDrive/ptbdb/preprocessed_data.pt",train_ids=train_ids, test_ids = test_ids, val_ids=[],train_ratio=0.8, val_ratio=0)
  model = CNN_1D()
  optimizer = optim.Adam(model.parameters(), lr=lr)
  criterion = nn.BCELoss()
  losses = train(model, criterion, optimizer, train_loader, train_loader ,epochs, device, return_loss=False) #train loader as val because we don't have val laoder
  metrics=eval(model, test_loader, device, all_metrics=True)
  print("Fianl metrics on test is: ",metrics )
  k_fold_acc[i] = metrics
  del model, optimizer, criterion
  del train_loader, val_loader, test_loader
  torch.cuda.empty_cache()

After filtering, we got: 228 records. Healthy: 80, Disease: 148
Load data from given path


Current loss in epoch 0 is 0.13805902004241943: 100%|██████████| 31/31 [00:07<00:00,  4.26it/s]


Acc on val in epoch 0 is: 0.6649870801033592


Current loss in epoch 1 is 0.03806470334529877: 100%|██████████| 31/31 [00:07<00:00,  4.25it/s]


Acc on val in epoch 1 is: 0.9984496124031008


...

Acc on val in epoch 5 is: 1.0


Current loss in epoch 6 is 0.000935537158511579: 100%|██████████| 31/31 [00:07<00:00,  4.13it/s]


Acc on val in epoch 6 is: 1.0


Current loss in epoch 7 is 0.00046167417895048857:  84%|████████▍ | 26/31 [00:06<00:01,  4.06it/s]


KeyboardInterrupt: 

In [43]:
k_fold_acc = {0: {'acc': 0.9265175718849841, 'prec': 0.8923533778767632, 'sens': 0.986863711001642, 'spec': np.float64(0.8509763617677287), 'f1': 0.9372319688109162}, 1: {'acc': 0.8871454435727217, 'prec': 0.981169474727453, 'sens': 0.8549222797927462, 'spec': np.float64(0.9619238476953907), 'f1': 0.9137055837563451}, 2: {'acc': 0.9739301524840138, 'prec': 0.9678832116788321, 'sens': 0.9932584269662922, 'spec': np.float64(0.9369627507163324), 'f1': 0.9804066543438078}, 3: {'acc': 0.9199594731509625, 'prec': 0.8916841369671559, 'sens': 0.9976544175136826, 'spec': np.float64(0.7769784172661871), 'f1': 0.9416974169741698}, 4: {'acc': 0.9315992292870906, 'prec': 0.9608333333333333, 'sens': 0.9238782051282052, 'spec': np.float64(0.9432367149758454), 'f1': 0.9419934640522876}}
save_file(object_to_save=k_fold_acc, save_name="k_fold_oldcnn_2.json")

{0: {'acc': 0.9265175718849841, 'prec': 0.8923533778767632, 'sens': 0.986863711001642, 'spec': np.float64(0.8509763617677287), 'f1': 0.9372319688109162}, 1: {'acc': 0.8871454435727217, 'prec': 0.981169474727453, 'sens': 0.8549222797927462, 'spec': np.float64(0.9619238476953907), 'f1': 0.9137055837563451}, 2: {'acc': 0.9739301524840138, 'prec': 0.9678832116788321, 'sens': 0.9932584269662922, 'spec': np.float64(0.9369627507163324), 'f1': 0.9804066543438078}, 3: {'acc': 0.9199594731509625, 'prec': 0.8916841369671559, 'sens': 0.9976544175136826, 'spec': np.float64(0.7769784172661871), 'f1': 0.9416974169741698}, 4: {'acc': 0.9315992292870906, 'prec': 0.9608333333333333, 'sens': 0.9238782051282052, 'spec': np.float64(0.9432367149758454), 'f1': 0.9419934640522876}}


In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Approximate the Compute and Memory Requirements

In [37]:
!pip install ptflops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [39]:
from ptflops import get_model_complexity_info
model_1 = CNN_2D()
model_2 = CNN_1D()
model_1.to(device)
model_2.to(device)
macs, params = get_model_complexity_info(model_1, (12, 5000), as_strings=True,print_per_layer_stat=False, verbose=False)
print(f"The 2D Conv based model requires {macs} macs and has {params} params")

macs, params = get_model_complexity_info(model_2, (12, 5000), as_strings=True,print_per_layer_stat=False, verbose=False)
print(f"The 1D Conv based model requires {macs} macs and has {params} params")


The 2D Conv based model requires 7.25 GMac macs and has 3.68 M params
The 1D Conv based model requires 132.71 MMac macs and has 1.24 M params
